# BFGS 

Implementacja BFGS.

In [1]:
using LinearAlgebra
using Zygote # różniczkowanie

## Algorytm Line Search

Do algorytmu BFGS będziemy potrzebować jakiegoś alogortymu line search. Na potrzeby tego projektu został wybrany algorytm Backtracking Line Search.gitbu

https://en.wikipedia.org/wiki/Backtracking_line_search

In [2]:
function BacktrackingLineSearch(f, x, p; α=0.99, τ=0.95, c=0.3, maxIterations=1000)::Float64
    m = f'(x)'p
    t = -c * m
    i = 0
    while i < maxIterations
        if f(x) - f(x + α * p) >= α * t
            break
        end
        α *= τ
        i += 1
    end
    return α
end

BacktrackingLineSearch (generic function with 1 method)

## BFGS: implementacja działająca i ładnie wyglądająca

Na początku imlementujemy BFGS tak, żeby był czytelny (używamy operacji macierzowych bez prealokacji).

In [3]:
function Bfgs(f, x::Vector{Float64}; maxIterations=1000, ϵ::Float64=1e-6, maxLineSearchIterations::Int64=10000)
    B = I # Tutaj lepsze zrobić lepsze wyszukanie
    ∇f = f'(x)
    i = 0
    while i < maxIterations
        p = B \ -∇f
        s = BacktrackingLineSearch(f, x, p; maxIterations=maxLineSearchIterations) * p
        
        if s's < ϵ
            break
        end

        x .+= s
        ∇fn = f'(x)
        y = ∇fn - ∇f
        ∇f = ∇fn
        Bs = B * s
        B += y * y' ./ y's - Bs * Bs' ./ s'Bs
        i += 1
    end
    return x
end

Bfgs (generic function with 1 method)

### Test BFGS

Testy przeprowadzone z pomocą funkcji Rosenbrocka

https://en.wikipedia.org/wiki/Rosenbrock_function



In [21]:
a = 1
b = 100
rossenbrock(x) = (a - x[1])^2 + b * (x[2] - x[1]^2)^2
rossenbrock([1,1])

0

In [22]:
Bfgs(rossenbrock, [1000.0, 402130.0]; ϵ=1e-30, maxIterations=Inf)
@time Bfgs(rossenbrock, [1000.0, 402130.0]; ϵ=1e-30, maxIterations=Inf)

  0.017393 seconds (278.84 k allocations: 12.012 MiB)


2-element Vector{Float64}:
 1.0
 1.0

Działa!

## Zomptymalizowane metody macierzowe wykorzystujace prealokację
Teraz zacznijmy optymalizować algorytm BFGS.
W tym celu zaimplementujemy kilka operacji macierzowych wykorzystujących prealokację których nie ma w LinearAlgebra.

In [6]:
function mulVectorByItsTranspose!(vector::Vector{Float64}, preallocatedMatrix::Matrix{Float64})::Matrix{Float64}
    preallocatedMatrix .*= 0
    for i in 1:length(vector)
        for j in 1:length(vector)
            preallocatedMatrix[i, j] = vector[i] * vector[j]
        end
    end
    return preallocatedMatrix
end        
    

mulVectorByItsTranspose! (generic function with 1 method)

In [7]:
y = [3.0, 5, 10, 20]
preallocatedMatrix = zeros(length(y), length(y))
mulVectorByItsTranspose!(y, preallocatedMatrix)
@time mulVectorByItsTranspose!(y, preallocatedMatrix)


  0.000000 seconds


4×4 Matrix{Float64}:
  9.0   15.0   30.0   60.0
 15.0   25.0   50.0  100.0
 30.0   50.0  100.0  200.0
 60.0  100.0  200.0  400.0

In [8]:
function mulVectors!(v::Vector{Float64}, y::Vector{Float64}, preallocatedMatrix::Matrix{Float64})::Matrix{Float64}
    n = length(v)
    for i = 1:n, j = 1:n
        preallocatedMatrix[i, j] = v[i] * y[j]
    end
    return preallocatedMatrix
end         
    
    

mulVectors! (generic function with 1 method)

In [9]:
x = [1.0, 2, 3]
y = [4.0, 5, 6]
preallocatedMatrix = zeros(3, 3)
mulVectors!(x, y, preallocatedMatrix)
@time mulVectors!(x, y, preallocatedMatrix)

  0.000000 seconds


3×3 Matrix{Float64}:
  4.0   5.0   6.0
  8.0  10.0  12.0
 12.0  15.0  18.0

In [10]:
function sub!(x, y, preallocated)
    preallocated .*= 0
    preallocated .+= x
    preallocated .-= y
    return preallocated
end


sub! (generic function with 1 method)

In [11]:
a = [ 1 2 3]
b = [ 4 5 6]
preaalocated = [ 9090 1412414 42412]
sub!(b, a, preaalocated)

preaalocated = [ 9090 1412414 42412]
@time sub!(b, a, preaalocated)

  0.000002 seconds


1×3 Matrix{Int64}:
 3  3  3

In [12]:
function add!(x, y, preallocated)
    preallocated .*= 0
    preallocated .+= x
    preallocated .+= y
    return preallocated
end


add! (generic function with 1 method)

In [13]:
a = [ 1 2 3]
b = [ 4 5 6]
preaalocated = [ 9090 1412414 42412]
add!(b, a, preaalocated)
preaalocated = [ 9090 1412414 42412]
@time add!(b, a, preaalocated)


  0.000002 seconds


1×3 Matrix{Int64}:
 5  7  9

In [14]:
function BfgsWithPreAllocation(f, x::Vector{Float64}; maxIterations=Inf, ϵ::Float64=1e-6, maxLineSearchIterations::Int64=10000)
	∇f = f'(x)
	B = zeros(length(x), length(x)) # I # Tutaj lepsze zrobić lepsze wyszukanie
    for i=1:length(x)
        B[i, i] = 1
    end
	i = 0
    n = length(x)
	y = zeros(n)
    yyT = zeros(n, n)
	BS = zeros(n, 1)
    BSBST = zeros(n, n)
    p = zeros(n)
    s = zeros(n)
	while i < maxIterations
		p = B \ -∇f # tu są 3 alokacje
         mul!(s, BacktrackingLineSearch(f, x, p; maxIterations=maxLineSearchIterations), p) #  Armijo ma dużo alokacji
		# s = BacktrackingLineSearch(f, x, p; maxIterations=maxLineSearchIterations) * p
		if s's < ϵ
			break
		end

		x .+= s
		∇fn = f'(x)
		sub!(∇fn, ∇f, y) # y = ∇fn - ∇f
		∇f = ∇fn
		mul!(BS, B, s)
		B .+= rdiv!(mulVectorByItsTranspose!(y, yyT), y's) 
        B .-=  mul!(BSBST, BS, BS') ./ s'BS
		i += 1
	end
	return x
end


BfgsWithPreAllocation (generic function with 1 method)

In [27]:
BfgsWithPreAllocation(rossenbrock, [1000.0, 402130.0]; ϵ=1e-30, maxIterations=Inf)
@time Bfgs(rossenbrock, [1000.0, 402130.0]; ϵ=1e-30, maxIterations=Inf)
@time BfgsWithPreAllocation(rossenbrock, [1000.0, 402130.0]; ϵ=1e-30, maxIterations=Inf)

  0.017556 seconds (278.84 k allocations: 12.012 MiB)
  0.016549 seconds (265.00 k allocations: 11.160 MiB)


2-element Vector{Float64}:
 1.0
 1.0000000000000002

Różnica w ilości alokacji jest niewielka ale istniejąca. 
Duża część alokacji (w zasadzie to większość) jest związana z różnicznkowaniem.

## Bibliografia

- https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm
- https://en.wikipedia.org/wiki/Backtracking_line_search

